### 1. Реалізувати підпрограми із необхідними математичними операціями: обчисленням оберненого елементу за модулем із використанням розширеного алгоритму Евкліда, розв’язуванням лінійних порівнянь. При розв’язуванні порівнянь потрібно коректно обробляти випадок із декількома розв’язками, повертаючи їх усі.

In [90]:
import math
from collections import Counter
import pandas as pd
import os

In [91]:
def rozshyrenyy_nsd(a, b):
    if b == 0:
        return a, 1, 0
    nsd, x1, y1 = rozshyrenyy_nsd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return nsd, x, y

def obernyty_mod(a, m):
    nsd, x, _ = rozshyrenyy_nsd(a, m)
    if nsd != 1:
        raise ValueError(f"{a} ne maie obernenoho elementa za modulem {m}")
    return x % m

def vyrishyty_liniyne_spivvidnoshennya(a, b, m):
    nsd, x, _ = rozshyrenyy_nsd(a, m)

    if b % nsd != 0:
        return 0

    a_skorot = a // nsd
    b_skorot = b // nsd
    m_skorot = m // nsd

    x0 = (obernyty_mod(a_skorot, m_skorot) * b_skorot) % m_skorot

    rozvyazky = [(x0 + k * m_skorot) % m for k in range(nsd)]
    return rozvyazky

print(vyrishyty_liniyne_spivvidnoshennya(2, 4, 6))


[2, 5]


### 2. За допомогою програми обчислення частот біграм, яка написана в ході виконання комп’ютерного практикуму №1, знайти 5 найчастіших біграм запропонованого шифртексту (за варіантом).

In [92]:
import math
from collections import Counter
import pandas as pd

def obchyslyty_chastoty_bigram(tekst):
    bigramy = [tekst[i:i+2].replace(' ', '_') for i in range(0, len(tekst) - 1, 2)]
    
    zahalna_kilkist_bigram = len(bigramy)
    kilkist_bigram = Counter(bigramy)

    chastoty = {bigram: kilkist / zahalna_kilkist_bigram for bigram, kilkist in kilkist_bigram.items()}
    
    return chastoty, kilkist_bigram, zahalna_kilkist_bigram

def obchyslyty_entropiyu(chastoty):
    entropiya = (-sum(chastota * math.log2(chastota) for chastota in chastoty.values() if chastota > 0))/2
    return entropiya

with open('09.txt', 'r', encoding='utf-8') as file:
    tekst = file.read().replace('\n','')

unikalni_symvoly = sorted(set(tekst.replace(' ', '_')))

chastoty_bigram, kilkist_bigram, zahalna_kilkist_bigram = obchyslyty_chastoty_bigram(tekst)

matrytsya_bigram = pd.DataFrame(0.0, index=unikalni_symvoly, columns=unikalni_symvoly)

for bigram, chastota in chastoty_bigram.items():
    if len(bigram) == 2:
        pershyy_symvol = bigram[0]
        druhyy_symvol = bigram[1]
        matrytsya_bigram.at[pershyy_symvol, druhyy_symvol] = chastota



entropiya_H2 = obchyslyty_entropiyu(chastoty_bigram)
top_5_bigram_chastota = kilkist_bigram.most_common(5)
print(f"Матриця частот біграм без перетинів успішно збережена в 'chastoty_bigram_bez_peretyniv.xlsx'.")
print(f"Ентропія H2: {entropiya_H2:.6f}")
print(top_5_bigram_chastota)



Матриця частот біграм без перетинів успішно збережена в 'chastoty_bigram_bez_peretyniv.xlsx'.
Ентропія H2: 4.076883
[('ээ', 74), ('вд', 57), ('чф', 56), ('цг', 56), ('гн', 56)]


### 3. Перебрати можливі варіанти співставлення частих біграм мови та частих біграм шифртексту (розглядаючи пари біграм із п’яти найчастіших). Для кожного співставлення знайти можливі кандидати на ключ (a,b) шляхом розв’язання системи (1).


In [93]:
def pary_naichastishykh_bigram(spysok):
    pary_bigram = []
    for i in range(len(spysok)):
        for j in range(i + 1, len(spysok)):
            para_pryama = (spysok[i], spysok[j])
            para_obernena = (spysok[j], spysok[i])
            pary_bigram.append(para_pryama)
            pary_bigram.append(para_obernena)
    return pary_bigram

top_5_bigram = [x[0] for x in top_5_bigram_chastota]

populyarni_pary_tekstu = pary_naichastishykh_bigram(top_5_bigram)
print(populyarni_pary_tekstu)
print(len(populyarni_pary_tekstu))

populyarni_pary_movy = pary_naichastishykh_bigram(['ст', 'но', 'то', 'на', 'ен'])
print(populyarni_pary_movy)
print(len(populyarni_pary_movy))


[('ээ', 'вд'), ('вд', 'ээ'), ('ээ', 'чф'), ('чф', 'ээ'), ('ээ', 'цг'), ('цг', 'ээ'), ('ээ', 'гн'), ('гн', 'ээ'), ('вд', 'чф'), ('чф', 'вд'), ('вд', 'цг'), ('цг', 'вд'), ('вд', 'гн'), ('гн', 'вд'), ('чф', 'цг'), ('цг', 'чф'), ('чф', 'гн'), ('гн', 'чф'), ('цг', 'гн'), ('гн', 'цг')]
20
[('ст', 'но'), ('но', 'ст'), ('ст', 'то'), ('то', 'ст'), ('ст', 'на'), ('на', 'ст'), ('ст', 'ен'), ('ен', 'ст'), ('но', 'то'), ('то', 'но'), ('но', 'на'), ('на', 'но'), ('но', 'ен'), ('ен', 'но'), ('то', 'на'), ('на', 'то'), ('то', 'ен'), ('ен', 'то'), ('на', 'ен'), ('ен', 'на')]
20


In [94]:
def pary_u_chysla(spysok, stan_pary=True):
    abetka = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
    chysla_pary = []
    if stan_pary:
        for para in spysok:
            x1 = abetka.index(para[0][0]) * 31 + abetka.index(para[0][1])
            x2 = abetka.index(para[1][0]) * 31 + abetka.index(para[1][1])
            chysla_pary.append((x1, x2))
    else:
        for elem in spysok:
            x1 = abetka.index(elem[0]) * 31 + abetka.index(elem[1])
            chysla_pary.append(x1)
    return chysla_pary

chysla_pary_tekstu = pary_u_chysla(populyarni_pary_tekstu)
chysla_pary_movy = pary_u_chysla(populyarni_pary_movy)

print(chysla_pary_tekstu)
print(chysla_pary_movy)


[(896, 66), (66, 896), (896, 733), (733, 896), (896, 685), (685, 896), (896, 106), (106, 896), (66, 733), (733, 66), (66, 685), (685, 66), (66, 106), (106, 66), (733, 685), (685, 733), (733, 106), (106, 733), (685, 106), (106, 685)]
[(545, 417), (417, 545), (545, 572), (572, 545), (545, 403), (403, 545), (545, 168), (168, 545), (417, 572), (572, 417), (417, 403), (403, 417), (417, 168), (168, 417), (572, 403), (403, 572), (572, 168), (168, 572), (403, 168), (168, 403)]


In [97]:
def znajty_kluchi(pary_bigram_movy, pary_bigram_shyfru, m):
    mod_m2 = m ** 2
    kandidaty = []
    
    for para_movy in pary_bigram_movy:
        for para_shyfru in pary_bigram_shyfru:
            x1, x2 = para_movy
            y1, y2 = para_shyfru
            
            delta_x = (x1 - x2) % mod_m2
            delta_y = (y1 - y2) % mod_m2

            a_kandydaty = vyrishyty_liniyne_spivvidnoshennya(delta_x, delta_y, mod_m2)

            if isinstance(a_kandydaty, list):
                for a in a_kandydaty:
                    b = (y1 - a * x1) % mod_m2
                    kandidaty.append((a, b))
            else:
                continue
    
    return kandidaty

m = 31

kluchi = znajty_kluchi(chysla_pary_movy, chysla_pary_tekstu, m)

for item in kluchi:
    print(f"Klyuch (a, b): {item}")
print(len(kluchi))


Klyuch (a, b): (502, 230)
Klyuch (a, b): (459, 732)
Klyuch (a, b): (219, 705)
Klyuch (a, b): (742, 924)
Klyuch (a, b): (820, 861)
Klyuch (a, b): (141, 720)
Klyuch (a, b): (802, 100)
Klyuch (a, b): (159, 902)
Klyuch (a, b): (678, 541)
Klyuch (a, b): (283, 258)
Klyuch (a, b): (318, 697)
Klyuch (a, b): (643, 54)
Klyuch (a, b): (300, 897)
Klyuch (a, b): (661, 236)
Klyuch (a, b): (601, 889)
Klyuch (a, b): (360, 529)
Klyuch (a, b): (583, 128)
Klyuch (a, b): (378, 711)
Klyuch (a, b): (943, 885)
Klyuch (a, b): (18, 867)
Klyuch (a, b): (459, 732)
Klyuch (a, b): (502, 230)
Klyuch (a, b): (742, 924)
Klyuch (a, b): (219, 705)
Klyuch (a, b): (141, 720)
Klyuch (a, b): (820, 861)
Klyuch (a, b): (159, 902)
Klyuch (a, b): (802, 100)
Klyuch (a, b): (283, 258)
Klyuch (a, b): (678, 541)
Klyuch (a, b): (643, 54)
Klyuch (a, b): (318, 697)
Klyuch (a, b): (661, 236)
Klyuch (a, b): (300, 897)
Klyuch (a, b): (360, 529)
Klyuch (a, b): (601, 889)
Klyuch (a, b): (378, 711)
Klyuch (a, b): (583, 128)
Klyuch (a, b): 

### 4. Для кожного кандидата на ключ дешифрувати шифртекст. Якщо шифртекст не є змістовним текстом російською мовою, відкинути цього кандидата.

In [96]:
import os
import numpy as np
from collections import Counter

def chastota_simvoliv(tekst, abetka):
    tekst = tekst.lower()
    kilkist_simvoliv = Counter([simvol for simvol in tekst if simvol in abetka])
    return kilkist_simvoliv

def chastota_bigram(tekst, abetka):
    bigramy = [tekst[i:i+2] for i in range(len(tekst)-1) if tekst[i:i+2] in abetka]
    kilkist_bigram = Counter(bigramy)
    return kilkist_bigram

def perevirka_tekstu(tekst, abetka):
    chastota_lyter = chastota_simvoliv(tekst, abetka)
    chastota_bigramu = chastota_bigram(tekst, abetka)

    naipopulyarnishi_lytery = sorted(chastota_lyter, key=chastota_lyter.get, reverse=True)[:5]
    naimensh_populyarni_lytery = sorted(chastota_lyter, key=chastota_lyter.get, reverse=True)[-5:]

    populyarni_lytery = {'о', 'а', 'е'}
    ridkisni_lytery = {'ф', 'щ', 'ь'}

    spivpaly_populyarnykh = len(set(naipopulyarnishi_lytery) & populyarni_lytery)
    spivpaly_ridkisnykh = len(set(naimensh_populyarni_lytery) & ridkisni_lytery)

    if spivpaly_populyarnykh >= 2 and spivpaly_ridkisnykh >= 1:
        populyarni_bigramy = ['ст', 'но', 'ен', 'то', 'на']
        top_bigramy = sorted(chastota_bigramu, key=chastota_bigramu.get, reverse=True)[:6]
        spivpaly_bigram = len(set(top_bigramy) & set(populyarni_bigramy))

        if spivpaly_bigram >= 2:
            return True
    
    return False

def tekst_u_chyslovyi_masiv(tekst, m):
    abetka = "абвгдежзийклмнопрстуфхцчшщьыэюя"
    rezultat = []
    for i in range(0, len(tekst) - 1, 2):
        if tekst[i].lower() in abetka and tekst[i + 1].lower() in abetka:
            x1 = abetka.index(tekst[i].lower())
            x2 = abetka.index(tekst[i + 1].lower())
            rezultat.append(x1 * m + x2)
    return rezultat

def chyslovyi_masiv_u_tekst(chyslovyi_masiv, m):
    abetka = "абвгдежзийклмнопрстуфхцчшщьыэюя"
    rezultat = []
    for num in chyslovyi_masiv:
        x1 = num // m
        x2 = num % m
        rezultat.append(abetka[x1] + abetka[x2])
    return ''.join(rezultat)

def rozshyfruvaty(chyslovyi_masiv, a, b, m):
    mod_m2 = m**2
    a_inv = obernyty_mod(a, mod_m2)
    return [(a_inv * (num - b)) % mod_m2 for num in chyslovyi_masiv]

with open('09.txt', 'r', encoding='utf-8') as file:
    shyfrtekst = file.read().strip().replace('\n', '')

m = 31
chyslovyi_shyfrtekst = tekst_u_chyslovyi_masiv(shyfrtekst, m)

output_dir = "rozshyfrovanі_teksty"
os.makedirs(output_dir, exist_ok=True)

for para in kluchi:
    a, b = para
    try:
        rozshyfrovanі_chysla = rozshyfruvaty(chyslovyi_shyfrtekst, a, b, m)
        rozshyfrovanij_tekst = chyslovyi_masiv_u_tekst(rozshyfrovanі_chysla, m)

        if perevirka_tekstu(rozshyfrovanij_tekst, "абвгдежзийклмнопрстуфхцчшщьыэюя"):
            output_filename = f"{output_dir}/rozshyfrovanij_a{a}_b{b}.txt"
            with open(output_filename, 'w', encoding='utf-8') as output_file:
                output_file.write(rozshyfrovanij_tekst)
            print(f"Rozshyfrovanij tekst zberezheno u fail: {output_filename}")
        else:
            print(f"Klyuch (a={a}, b={b}) ne proishov perevirku.")
    except ValueError as e:
        print(f"Propushcheno klyuch (a={a}, b={b}) - {e}")


Klyuch (a=502, b=230) ne proishov perevirku.
Klyuch (a=459, b=732) ne proishov perevirku.
Klyuch (a=219, b=705) ne proishov perevirku.
Klyuch (a=742, b=924) ne proishov perevirku.
Klyuch (a=820, b=861) ne proishov perevirku.
Klyuch (a=141, b=720) ne proishov perevirku.
Klyuch (a=802, b=100) ne proishov perevirku.
Rozshyfrovanij tekst zberezheno u fail: rozshyfrovanі_teksty/rozshyfrovanij_a159_b902.txt
Klyuch (a=678, b=541) ne proishov perevirku.
Klyuch (a=283, b=258) ne proishov perevirku.
Rozshyfrovanij tekst zberezheno u fail: rozshyfrovanі_teksty/rozshyfrovanij_a318_b697.txt
Klyuch (a=643, b=54) ne proishov perevirku.
Klyuch (a=300, b=897) ne proishov perevirku.
Klyuch (a=661, b=236) ne proishov perevirku.
Klyuch (a=601, b=889) ne proishov perevirku.
Klyuch (a=360, b=529) ne proishov perevirku.
Klyuch (a=583, b=128) ne proishov perevirku.
Klyuch (a=378, b=711) ne proishov perevirku.
Klyuch (a=943, b=885) ne proishov perevirku.
Klyuch (a=18, b=867) ne proishov perevirku.
Klyuch (a=45